# Globus Automate Introduction
## Tyler-edited version for Xtract. 

This notebook goes over using Automate to create a simple flow and monitor its status.

To use the notebook you will need to both install the SDK:

- pip install globus-automate-client

And be added to the Automate whitelist group to be allowed to create and run flows. Please email rchard@anl.gov to get access to this.

In [5]:
import sys
import os
import time
import json
sys.path.append(".")
os.environ['JUPYTER_PATH'] = '.'
CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

from globus_automate_client import  (get_access_token_for_scope, create_action_client, 
                                    create_flows_client)

# Globus Automate: Flows and Actions

## Flow Definition

* Flows are composed of *Action* invocations
* Each Action invocation reads from and contributes back to the *Flow State* which is referenced in Flow steps using the `InputPath` and `ResultPath` properties of an Action.

In [6]:
flow_definition = {
  "Comment": "Xtract: Crawl and Extract",
  "StartAt": "Transfer1",
  "States": {
    "Transfer1": {
      "Comment": "Crawl and Extraction of Data stored at user-defined Globus directory path",
      "Type": "Action",
        "ActionUrl": "https://xtract.materialsdatafacility.org",
        "ActionScope": "https://auth.globus.org/scopes/34284fb1-2eea-4532-a04a-9c8ad1702856/xtract_crawl_and_extract",
      "InputPath": "$.Xtract1Input",
      "ResultPath": "$.Xtract1Result",
      "WaitTime": 3600,
      "End": True
    }
  }
}

* This flow composes two transfers into a single logical operation
  * Suitable, for example, for doing a two stage transfer between a local campus endpoint, a DMZ data transfer endpoint, and a dataset repository.
  * Each step in the Flow uses the same Action: Transfer which is referenced by URL
  * Globus Auth Scope information is required to authenticate operations to the Action.
  * Source and destination information for the Transfer state are given in `InputPath` and `ResultPath`
    * Format of the input is Action dependent (see below)


In [7]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title='test-two-hop')
flow_id = flow['id']
print(flow)
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'administered_by': [], 'api_version': '1.0', 'definition': {'Comment': 'Xtract: Crawl and Extract', 'StartAt': 'Transfer1', 'States': {'Transfer1': {'ActionScope': 'https://auth.globus.org/scopes/34284fb1-2eea-4532-a04a-9c8ad1702856/xtract_crawl_and_extract', 'ActionUrl': 'https://xtract.materialsdatafacility.org', 'Comment': 'Crawl and Extraction of Data stored at user-defined Globus directory path', 'End': True, 'InputPath': '$.Xtract1Input', 'ResultPath': '$.Xtract1Result', 'Type': 'Action', 'WaitTime': 3600}}}, 'description': '', 'globus_auth_scope': 'https://auth.globus.org/scopes/9606f1aa-4579-4e0a-ad28-f13ea9601431/flow_9606f1aa_4579_4e0a_ad28_f13ea9601431', 'globus_auth_username': '9606f1aa-4579-4e0a-ad28-f13ea9601431@clients.auth.globus.org', 'id': '9606f1aa-4579-4e0a-ad28-f13ea9601431', 'keywords': [], 'log_supported': True, 'principal_urn': 'urn:globus:auth:identity:9606f1aa-4579-4e0a-ad28-f13ea9601431', 'runnable_by': [], 'subtitle': '', 'synchronous': F

In [8]:
flow_input = {
  "Xtract1Input": {
    "source_endpoint_id": "go#ep1",
    "destination_endpoint_id": "go#ep2",
    "eid": "e38ee745-6d04-11e5-ba46-22000b92c6ec",
    "dir_path": "/MDF/mdf_connect/prod/data",
    "grouper": "matio",
    "transfer_items": [
      {
        "source_path": "/~/core",
        "destination_path": "/~/globus_publish_2.txt",
        "recursive": False
      }
    ]
  }
}

flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
print(flow_action)

flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(2)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')
print (flow_action)


Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=e6c75d97-532a-4c88-b031-8584a319fa3e&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2F9606f1aa-4579-4e0a-ad28-f13ea9601431%2Fflow_9606f1aa_4579_4e0a_ad28_f13ea9601431&state=_default&response_type=code&code_challenge=v9Vp3xJ_EC9KqUbjm8lLadEPAtQYrd5OiOLuzbGzztA&code_challenge_method=S256&access_type=offline&prefill_named_grant=Globus+Automate+Client


Note that this link can only be used once! If login or a later step in the flow fails, you must restart it.
Enter resulting code:orJyFQFmZyLXF7NALeOmDanagxrfoL
GlobusHTTPResponse({'action_id': '04454f38-73d7-4151-9fe7-74ba2b877265', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:f12d8fb3-5502-4895-ac33-e59581f9f03f', 'details': {'code': 'ActionStarted', 'description': 'State Transfer1 of type Action started', 'details': {'input': {'Xtract1Input': {'destination_endpoint_id': 'go#ep2', 'dir_pat